In [81]:
import numpy as np
import cv2 as cv
import matech_utilities as mu
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20, 15)


In [89]:
cap = cv.VideoCapture(0) # cv.VideoCapture("images/german.mp4")
# r, h, c, w = 220, 340, 680, 180  # calculator
# r, h, c, w = 270, 250, 720, 100 # tight
r, h, c, w = 75, 450, 825, 350  # german
track_window = (c, r, w, h)
org_box = np.array([[c, r, 1],
                    [c, r + h, 1],
                    [c + w, r + h, 1],
                    [c + w, r, 1]
                    ])

# params for ShiTomasi corner detection
feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)
# Parameters for lucas kanade optical flow
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (1500, 3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
org_mask = mu.create_mask(old_frame)
print(org_mask.shape, org_mask.dtype)
org_mask[r:r + h, c:c + w] = 255
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
# p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
_, p00, _ = mu.extract_features(old_frame, resize=False, mask=org_mask)
p0 = np.array([p.pt for p in p00], dtype=np.float32).reshape((-1, 1, 2))
prev_box = org_box
# print(p0.dtype, p0.dtype)
# print(type(p00))
mu.cv_show_pics(1,1,[mu.draw_points(p0.reshape((-1, 2)), old_frame)])

AttributeError: 'NoneType' object has no attribute 'shape'

In [87]:
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

fourcc = cv.VideoWriter_fourcc(*'DIVX')
vh, vw, _ = old_frame.shape
out = cv.VideoWriter('output.mkv', fourcc, 25.0, (vw, vh), isColor=True)
while True:
    ret, frame = cap.read()
    if ret == False:
        break

    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(
        old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    print(p1.shape, st.shape)
    condition = np.array(st == 1)  
    good_new = p1[condition]
    good_old = p0[condition]
    h, m = cv.findHomography(good_old, good_new, cv.RANSAC, 1.0)
    box = mu.transform_with_homography(h, prev_box)
    res, center = mu.draw_box_homogeneous(box, frame, True)
    # draw the tracks
    for (a, b), c in zip(good_new, m):
        if c == 1:
            frame = cv.circle(frame, (a, b), 5, color[i].tolist(), -1)
    img = cv.add(frame, mask)
    out.write(img)
    # mu.cv_show_pics(1,1,[img])
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
    prev_box = box
    
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

AttributeError: 'NoneType' object has no attribute 'shape'

In [90]:
out.release()
cap.release()


In [74]:
h, mask = cv.findHomography(good_old, good_new, cv.RANSAC, 1.0)

In [75]:
mask

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]], dtype=uint8)

In [92]:
fake = np.array([[0,0],[0,1],[1,0],[1,1]])
h, m = cv.findHomography(fake, fake, 0, 5.0)
print(h)

[[  1.00000000e+00   0.00000000e+00  -9.61481343e-17]
 [  0.00000000e+00   1.00000000e+00  -2.88444403e-16]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]
